# Systemic Risk Dashboard - A Data Science Approach to Predict the Impact of Collateralization on Systemic Risk 
# Sistemik Risk Panosu - Teminatlandırmanın Sistemik Risk Üzerindeki Etkisini Tahmin Etmek İçin Bir Veri Bilimi Yaklaşımı

In [2]:
import sys
import math
import json

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

from bqplot import *

import networkx as nx
import visJS2jupyter.visJS_module

from ipywidgets import Dropdown, Text, HBox, Button, VBox, RadioButtons, Layout
from IPython.display import display, clear_output

from webcolors import rgb_percent_to_hex

In [3]:
# Change notebook container width to full-screen (100%)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_6116\1285390570.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
df = pd.read_csv('df_exposures.csv')
with open('nodes.json') as fp:
    nodes = json.load(fp)
with open('edges.json') as fp:
    edges = json.load(fp)

regulations = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized']
financial_system_ids = df['FS'].unique().tolist()

In [6]:
def load_data(seed=42):
    np.random.seed(seed)
    df = pd.read_csv('df_exposures.csv')
    with open('nodes.json') as fp:
        nodes = json.load(fp)
    with open('edges.json') as fp:
        edges = json.load(fp)

    regulations = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized']
    financial_system_ids = df['FS'].unique().tolist()
    return df, nodes, edges, regulations, financial_system_ids

class Model():
    def __init__(self, df, nodes, edges, regulations, financial_system_ids):
        self.df = df
        self.nodes = nodes
        self.edges = edges
        self.regulations = regulations
        self.financial_system_ids = financial_system_ids
        self.menu = Menu(regulations, financial_system_ids)
        self.w = None
  
    def draw_network(self):
        reg = self.menu.reg_selector.value
        fs_id = self.menu.fs_selector.value
        G_nodes = self.nodes[reg][fs_id]
        G_edges = self.edges[reg][fs_id]
        
        graph_type = self.menu.graph_type
        G = nx.Graph()
        G.add_edges_from(G_edges)
        pos = nx.spring_layout(G)
        nodes_dict = [{'id':'CP_' + str(n[0]),
                       'node_label': 'CP_' + str(n[0]),
                        'x':pos[n[0]][0]*1000,
                        'y':pos[n[0]][1]*1000,
                        'node_size':n[1]['BaselEEPE+']/100000,
                        'color': mpl.colors.rgb2hex(cm.get_cmap('bwr')(n[1]['BaselEEPE_rho+'])),
                       } for n in G_nodes] 

        node_map = dict(zip([n[0] for n in G_nodes], range(len(G_nodes)))) 
        edges_dict = [{'source':node_map[G_edges[i][0]], 
                       'target':node_map[G_edges[i][1]], 
                       'edge_width': math.log10(G_edges[i][2]['BaselEEPE']/10000 + 1),  # try taking log10()
                              } for i in range(len(G_edges))]

        if graph_type.value == 'Risk Graph':
            self.w = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                      node_size_multiplier=0.5,
                                      node_size_transform = '',
                                      node_size_field='node_size',
                                      node_label_field='node_label',
                                      node_scaling_label_enabled=False,
                                      node_color_highlight_border='red',
                                      node_color_highlight_background='#D3918B',
                                      node_color_hover_border='blue',
                                      node_color_hover_background='#8BADD3',
                                      node_font_size=25,
                                      edge_width_field='edge_width',
                                      edge_arrow_to=True,
                                      physics_enabled=True,
                                      edge_color_highlight='#8A324E',
                                      edge_color_hover='#8BADD3',
                                      edge_width=3,
#                                       edge_width='edge_width',
                                      max_velocity=15,
                                      min_velocity=1,
                                      graph_height=430,
                                      graph_width='40%',
                                      edge_smooth_enabled = True
                                                        )

            self.w.data = self.w.data.replace("""height="1200px" width="100%""", """height="auto" width="40%""") # change size of canvas
            display(self.w)

        else:
            self.w = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                                                physics_enabled=True,
                                                                graph_height=430,
                                                                graph_width='40%',
                                                                max_velocity=15,
                                                                min_velocity=1,
                                                             )
            display(self.w)
        
class Menu():
    def __init__(self, regulations, financial_system_ids):
        self.regulations = regulations
        self.financial_system_ids = financial_system_ids
        
        self.reg_text = Text("Regulatory Regime:")
        self.reg_selector = Dropdown(options = regulations) 
        self.fs_text = Text("Financial System:")
        self.fs_selector = Dropdown(options = financial_system_ids)
        self.graph_type = RadioButtons(options=['Trade Relations', 'Risk Graph'],layout=Layout(width='20%', height='80px'), value='Risk Graph')
        self.draw_button = Button(description="Load")
        self.is_menu_displayed = False
    
    def show(self):      
        if self.is_menu_displayed == False: 
            display(HBox([self.graph_type, VBox([self.fs_text, self.fs_selector]), VBox([self.reg_text, self.reg_selector]), self.draw_button]))
            self.is_menu_displayed = True

In [7]:
df, nodes, edges, regulations, financial_system_ids = load_data()
model = Model(df, nodes, edges, regulations, financial_system_ids)

In [11]:
# bqplot graph 1
x1 = model.regulations
df_total1 = model.df[model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].sum()
y1 = df_total1.to_numpy()/1000000


xs1 = OrdinalScale()
ys1 = LinearScale()

bar1 = Bars(x=x1, y=y1, 
            scales={'x': xs1, 'y': ys1}, 
            colors=['#1f77b4', '#ff7f0e', '#2ca02c'],
            padding=0.2
           )
xax1 = Axis(scale=xs1)
yax1 = Axis(scale=ys1,
            label='EEPE+ in USD (in millions)', 
            label_offset='47px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid'
           )
fig_layout = Layout(width='50%',
                    display='flex',
                    align_items='stretch',
                    justify_content='flex-start',
                    max_aspect_ratio=100,
                   )

fig1 = Figure(title='Impact of Collateralization on Systemic Risk', 
              marks=[bar1], 
              axes=[xax1, yax1], 
              layout=fig_layout,
              animation_duration=1000
             )

In [12]:
# bqplot graph 2

x2 = ['CP_{}'.format(x) for x in range (0, 14)]
df_eepep = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
y2 = df_total1.to_numpy()/1000000

xs2 = OrdinalScale()
ys2 = LinearScale(min=0)

bar2 = Bars(x=x2, y=y2, 
            type='grouped',
            scales={'x': xs2, 'y': ys2}, 
            colors=['#1f77b4', '#ff7f0e', '#2ca02c'],
            labels=model.regulations,
            display_legend=True,
            padding=0.2)

xax2 = Axis(scale=xs2)

yax2 = Axis(scale=ys2,
            label='EEPE+ in USD (in millions)', 
            label_offset='47px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid')
fig2 = Figure(title='Impact of Collateralization on Systemic Risk By Counterparty', 
              marks=[bar2], 
              axes=[xax2, yax2], 
              legend_location='top-right',
              layout=fig_layout,
              animation_duration=1000
             )

In [14]:
# bqplot graph 3

x3 = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized', '     '] # hacky legend stuff, should change in final to be model.regulations perhaps

df_eepep_rhop = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE_rho+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
df_eepep_rhop[df_eepep_rhop.columns] = df_eepep_rhop[df_eepep_rhop.columns].div(df_eepep_rhop[df_eepep_rhop.columns].sum(axis=0), axis=1).multiply(100)
df_eepep_rhop['     '] = 0 #five spaces
y3 = df_eepep_rhop.to_numpy()

xs3 = OrdinalScale()
ys3 = LinearScale(min=0)
cs3 = OrdinalColorScale(colors=['Red','Blue'])

NUM_COLORS3 = 15
CMAP3 = cm.get_cmap('tab20c')
colors3 = [CMAP3(1.*i/NUM_COLORS3) for i in range(0,NUM_COLORS3)]

colors3_hex = [rgb_percent_to_hex(((str(color[0]*100)+'%'), (str(color[1]*100)+'%'), (str(color[2]*100)+'%'))) for color in colors3]

bar3 = Bars(x=x3, y=y3, 
            scales={'x': xs3, 
                    'y': ys3,
                    'color' : cs3
                   },
            labels=['CP_{}'.format(x) for x in range(0, NUM_COLORS3)],
            colors=colors3_hex,
            display_legend=True,
            padding=0.2
           )

xax3 = Axis(scale=xs3)
yax3 = Axis(scale=ys3,
            label='Rho+ In %', 
            label_offset='49px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid'
           )

fig3 = Figure(title='Concentration of Risk', 
              marks=[bar3], 
              axes=[xax3, yax3], 
              legend_location='top-right',
              layout=fig_layout,
              animation_duration=1000, 
             )

In [15]:
figsw1_list = [Figure(layout=Layout(width='50%')), fig1]
display(widgets.HBox(figsw1_list, align_content = 'stretch'))

In [16]:
figs23_list = [fig2, fig3]
display(widgets.HBox(figs23_list, align_content = 'stretch'))

In [17]:
clear_output(wait=True)
model.menu.show()

In [18]:
from IPython.display import Javascript
from ipywidgets import widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
model.menu.draw_button.on_click(run_all)

In [115]:
### ALL CODE BELOW WILL BE EXECUTED BY 'LOAD' BUTTON ###
### AŞAĞIDAKİ TÜM KODLAR 'YÜKLE' DÜĞMESİ TARAFINDAN ÇALIŞTIRILACAKTIR ###

In [19]:
model.draw_network()

In [22]:
# bqplot graph 1 update:
df_total1 = model.df[model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].sum()
y1 = df_total1.to_numpy()/1000000
bar1.y = y1

In [24]:
# bqplot graph 2 update:
df_eepep = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
y2 = df_total1.to_numpy()/1000000
bar2.y = y2

In [26]:
# bqplot graph 3 update:
df_eepep_rhop = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE_rho+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
df_eepep_rhop[df_eepep_rhop.columns] = df_eepep_rhop[df_eepep_rhop.columns].div(df_eepep_rhop[df_eepep_rhop.columns].sum(axis=0), axis=1).multiply(100)

# hacky adding empty column on right side for legend
# hacky, efsane için sağ tarafa boş sütun ekliyor
df_eepep_rhop['     '] = 0
y3 = df_eepep_rhop.to_numpy()

bar3.y = y3